In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

C:\Users\anees\AppData\Local\Temp\ipykernel_23064\3748546470.py:1: DtypeWarning: Columns (54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])


In [3]:
df.columns

Index(['p_match', 'inns', 'bat', 'p_bat', 'team_bat', 'bowl', 'p_bowl',
       'team_bowl', 'ball', 'ball_id', 'outcome', 'score', 'out', 'dismissal',
       'p_out', 'over', 'ball.1', 'noball', 'wide', 'byes', 'legbyes',
       'cur_bat_runs', 'cur_bat_bf', 'cur_bowl_ovr', 'cur_bowl_wkts',
       'cur_bowl_runs', 'inns_runs', 'inns_wkts', 'inns_balls',
       'inns_runs_rem', 'inns_balls_rem', 'inns_rr', 'inns_rrr', 'target',
       'max_balls', 'date', 'year', 'ground', 'country', 'winner', 'toss',
       'competition', 'bat_hand', 'bowl_style', 'bowl_kind', 'batruns',
       'ballfaced', 'bowlruns', 'bat_out', 'wagonX', 'wagonY', 'wagonZone',
       'pitchLine', 'pitchLength', 'shotType'],
      dtype='object')

In [4]:
df.head()

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,batruns,ballfaced,bowlruns,bat_out,wagonX,wagonY,wagonZone,pitchLine,pitchLength,shotType
0,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,1,1001349_1_1_1,...,0,1,0,True,208,178,3,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED
1,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,2,1001349_1_1_2,...,0,1,0,True,243,175,3,ON_THE_STUMPS,GOOD_LENGTH,DEFENDED
2,1001349,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,1001349_1_1_3,...,1,1,1,True,189,242,4,ON_THE_STUMPS,SHORT_OF_A_GOOD_LENGTH,DEFENDED
3,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,3,1001351_1_1_3,...,0,1,0,True,166,194,5,OUTSIDE_OFFSTUMP,SHORT_OF_A_GOOD_LENGTH,DEFENDED
4,1001351,1,Aaron Finch,5334,Australia,Lasith Malinga,49758,Sri Lanka,4,1001351_1_1_4,...,4,1,4,True,15,222,6,OUTSIDE_OFFSTUMP,GOOD_LENGTH,CUT_SHOT


In [5]:
df[(df['ball']>11)][['team_bat','team_bowl','inns','ball','over','date','competition']]

,team_bat,team_bowl,inns,ball,over,date,competition
165211,Titans,Dolphins,1,12,17,2016-11-20,CSA T20 Challenge
328978,Dolphins,Lions,1,13,17,2015-11-04,Ram Slam T20 Challenge
328979,Dolphins,Lions,1,12,18,2015-11-04,Ram Slam T20 Challenge
428555,Panadura Sports Club,Sri Lanka Army,1,12,4,2021-03-07,SLC T20
622199,Kurunegala Youth Cricket Club,Sri Lanka Navy Sports Club,2,12,1,2022-06-12,Clubs T20
...,...,...,...,...,...,...,...
1434990,Sebastianites Cricket and Athletic Club,Colts Cricket Club,1,23,8,2023-06-07,Clubs T20
1434991,Sebastianites Cricket and Athletic Club,Colts Cricket Club,1,24,8,2023-06-07,Clubs T20
1505956,Malaysia,Hong Kong,1,12,4,2023-09-19,T20I
1507031,Papua New Guinea,Malaysia,2,12,1,2023-09-20,T20I


In [6]:
df[(df['ball']>11)].to_csv('uncleandata.csv')

These very low level T20 leagues, the data quality is often not reliable. Hence, this should explain very high ball values.

In [7]:
df=df[(df['ball']<=11)]

In [8]:
df.shape

(1554887, 55)

In [9]:
df[df['ball']!=df['ball.1']]

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,batruns,ballfaced,bowlruns,bat_out,wagonX,wagonY,wagonZone,pitchLine,pitchLength,shotType


ball.1 is duplicate of attribute ball.

In [10]:
df=df.drop(columns='ball.1')
df.shape

(1554887, 54)

In [11]:
df['bowl_style'].unique()

array(['RF', 'RFM', 'LB', 'LWS', 'RMF', 'SLA', 'OB', 'LBG', 'LFM', 'LF',
       'RM', 'LMF', 'RM/OB', 'LM', 'LFM/SLA', 'OB/LB', 'LM/SLA/LWS',
       'OB/SLA', 'RAB', 'LS', '-', 'LSM', 'RSM', 'RMF/OB', 'RFM/OB',
       'OB/LBG', 'RFM/LBG', 'RM/LB', 'RM/LBG', 'LMF/RM', 'RM/RSM',
       'SLA/LWS', 'LAB', 'RMF/LB', 'RFM/LB', 'RM/OB/LB', 'RS'],
      dtype=object)

In [12]:
df.iloc[:1000].to_excel('t20sample_data.xlsx', index=False)

In [13]:
unique_styles = ['RFM', 'RMF', 'LBG', 'LB', 'LS', 'RAB', 'LAB', 'OB', 'SLA', 'RAB','RSM','LSM']
result = pd.concat([df[df['bowl_style'] == style][['bowl','bowl_style']].drop_duplicates().head(2) for style in unique_styles])
result

,bowl,bowl_style
2276,Nuwan Kulasekara,RFM
3078,Vikum Sanjaya,RFM
7715,Asela Gunaratne,RMF
28810,Andrew Tye,RMF
20353,Fawad Ahmed,LBG
40125,Adam Zampa,LBG
3598,Seekkuge Prasanna,LB
55231,Cameron Boyce,LB
715323,Mohammed Aslam,LS
1100250,Manson Chikowero,LS


There are few bowling types which have ambiguous values. So after researching the bowlers, we came to know:
1. RFM, RMF means Right Arm Medium Fast
2. LBG, LB means Leg Break, which is usually right arm wrist spin bowlers
3. LS, SLA means Slow Left Arm Offie
4. OB means Right Arm Offie
5. RAB,LAB means Right/Left Arm Bowler, who is likely a part timer who don't have any particular style. Thus we are converting them to RSM(right arm slow medium) and LSM(left arm slow medium). 
Now we are changing these values into more understandable format.

In [14]:
df['bowl_style'] = df['bowl_style'].replace(['RFM', 'RMF'], 'RFM')
df['bowl_style'] = df['bowl_style'].replace(['LBG', 'LB'], 'RWS')
df['bowl_style'] = df['bowl_style'].replace(['LS', 'SLA'], 'SLA')
df['bowl_style'] = df['bowl_style'].replace(['RAB'], 'RSM')
df['bowl_style'] = df['bowl_style'].replace(['LAB'], 'LSM')
unique_styles = ['LFM/SLA', 'OB/LB','LM/SLA/LWS','OB/SLA','RMF/OB', 'RFM/OB', 'OB/LBG', 'RFM/LBG', 'RM/LB',
       'RM/LBG', 'LMF/RM', 'RM/RSM', 'SLA/LWS', 'RMF/LB', 'RFM/LB',
       'RM/OB/LB']
replacements = {
    'RFM': 'RFM',
    'RMF': 'RFM',
    'LBG': 'RWS',
    'LB': 'RWS',
    'LS': 'SLA',
    'SLA': 'SLA',
    'RAB': 'RS',
    'LAB': 'LS'
}

# Iterate over unique styles and apply replacements
for style in unique_styles:
    replaced_style = '/'.join([replacements.get(part, part) for part in style.split('/')])
    df['bowl_style'] = df['bowl_style'].replace([style], replaced_style)
df['bowl_style'].unique()

array(['RF', 'RFM', 'RWS', 'LWS', 'SLA', 'OB', 'LFM', 'LF', 'RM', 'LMF',
       'RM/OB', 'LM', 'LFM/SLA', 'OB/RWS', 'LM/SLA/LWS', 'OB/SLA', 'RSM',
       '-', 'LSM', 'RFM/OB', 'RFM/RWS', 'RM/RWS', 'LMF/RM', 'RM/RSM',
       'SLA/LWS', 'RM/OB/RWS', 'RS'], dtype=object)

In [15]:
df.describe()

,p_match,inns,p_bat,p_bowl,ball,score,p_out,over,noball,wide,...,inns_rrr,target,max_balls,year,batruns,ballfaced,bowlruns,wagonX,wagonY,wagonZone
count,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,...,734761.000000,734761.000000,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06,1.554887e+06
mean,1.189248e+06,1.472549e+00,5.063180e+05,5.669501e+05,3.620968e+00,1.278893e+00,5.063946e+05,1.002362e+01,5.714242e-03,3.919577e-02,...,10.341222,160.467487,8.801454e+01,2.019634e+03,1.208241e+00,9.675295e-01,1.253151e+00,8.543081e+01,8.985982e+01,2.142561e+00
std,1.508010e+05,4.992461e-01,3.745934e+05,3.755675e+05,1.818188e+00,1.574937e+00,3.746332e+05,5.640178e+00,9.165568e-02,2.557577e-01,...,14.148465,34.966115,5.238543e+01,2.554422e+00,1.586408e+00,1.772462e-01,1.579413e+00,1.103071e+02,1.100803e+02,2.660337e+00
min,7.223350e+05,1.000000e+00,4.068000e+03,4.068000e+03,1.000000e+00,0.000000e+00,4.068000e+03,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,2.015000e+03,-4.000000e+00,0.000000e+00,-3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.125649e+06,1.000000e+00,2.498660e+05,2.976300e+05,2.000000e+00,0.000000e+00,2.498660e+05,5.000000e+00,0.000000e+00,0.000000e+00,...,6.720000,140.000000,0.000000e+00,2.018000e+03,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.213058e+06,1.000000e+00,4.228760e+05,4.937730e+05,4.000000e+00,1.000000e+00,4.229650e+05,1.000000e+01,0.000000e+00,0.000000e+00,...,8.620000,161.000000,1.200000e+02,2.020000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.304057e+06,2.000000e+00,7.235730e+05,8.376110e+05,5.000000e+00,1.000000e+00,7.243810e+05,1.500000e+01,0.000000e+00,0.000000e+00,...,10.880000,182.000000,1.200000e+02,2.022000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.780000e+02,1.890000e+02,4.000000e+00
max,1.412550e+06,2.000000e+00,1.414161e+06,1.413607e+06,1.100000e+01,8.000000e+00,1.414161e+06,2.100000e+01,6.000000e+00,6.000000e+00,...,1002.000000,315.000000,3.000000e+02,2.023000e+03,7.000000e+00,1.000000e+00,8.000000e+00,4.150000e+02,4.660000e+02,8.000000e+00


We can see that the attributes batruns and bowlruns has negative values which is the errors in the data. These data should be removed

In [16]:
df=df[(df['batruns']>=0) & (df['bowlruns']>=0)]
df.shape

(1554671, 54)

Understanding WagonX, WagonY and WagonZone attributes

In [17]:
df.groupby(['wagonZone']).agg({'wagonX':'mean','wagonY':'mean'}).reset_index()

,wagonZone,wagonX,wagonY
0,0,0.000000,0.000000
1,1,228.723680,66.956557
2,2,279.865352,126.732172
3,3,284.986903,212.424818
4,4,221.928538,270.105581
5,5,137.538450,268.838547
6,6,84.483881,207.445834
7,7,86.823447,128.386469
8,8,151.927968,93.099248


In [19]:
df_rankings=pd.read_csv("T20ranking.csv")
df_rankings['Team']

0            India
1          England
2        Australia
3      New Zealand
4         Pakistan
5     South Africa
6      West Indies
7        Sri Lanka
8       Bangladesh
9      Afghanistan
10         Ireland
11         Namibia
12        Zimbabwe
13        Scotland
14     Netherlands
15           Nepal
16             UAE
17            Oman
18             PNG
19          Canada
Name: Team, dtype: object

In [ ]:
df_rankings['Team']=df_rankings['Team'].replace('UAE','United Arab Emirates').replace('PNG','Papua New Guinea')
df_rankings

In [ ]:
df['competition'].unique()

In [ ]:
relevant_competitions=['T20I', 'IPL', 'AUS v SL','T20 Blast','LPL', 'CPL', 'PSL', 'Big Bash League', 'BBL',
       "Men's Hundred",'MSL', 'MSL 2018','Super Smash','National T20 Cup', 'National T20', 'PAK-A v NZ-A T20s',
       'CSA T20 Challenge','CSA Provincial T20 Challenge', 'Syed Mushtaq Ali Trophy','SMA TROPHY','SMA TROPHY FINAL',
        'ENG v IND',  'CSA T20', 'Provincial T20','Africa T20 Cup',
       'PRO T20', 'Ireland A in Bangladesh unofficial T20I Series', 
       'Inter Pro T20', 'Hong Kong tour of Namibia','Ireland A in Namibia T20s', 'SA20',
       'PSL 2023', 'IPL 2023', 'CPL 2023']

In [ ]:
df.shape

In [ ]:
df=df[df['competition'].isin(relevant_competitions)]
df.shape

In [ ]:
df=df[(((df['competition']=='T20I') & (df['team_bat'].isin(df_rankings['Team'])) & (df['team_bowl'].isin(df_rankings['Team']))) | (df['competition'].isin(relevant_competitions[1:])))]
df.shape

# Batter's performaces against every bowling type

In [ ]:
total_runs=df.groupby(['p_bat','bat','bowl_style']).agg({'p_match':'nunique','batruns':'sum','ballfaced':'sum'}).reset_index().sort_values(by='batruns',ascending=False)
total_runs['strike rate']=total_runs['batruns']*100/total_runs['ballfaced']
total_runs.rename(columns={'p_match':'matches'},inplace=True)
total_runs.head()

In [ ]:
total_outs=df[df['out']==True].groupby(['p_bat','bat','bowl_style']).size().reset_index(name='count').sort_values(by='count',ascending=False)
total_outs.rename(columns={'count':'dismissals'},inplace=True)
total_outs.head()

In [ ]:
merged_df=pd.merge(total_runs,total_outs,on=['p_bat','bat','bowl_style'])
merged_df.columns

In [ ]:
merged_df['average']=merged_df['batruns']/merged_df['dismissals']
merged_df=merged_df[merged_df['ballfaced']>100].sort_values(by='average',ascending=False)
merged_df.shape

In [ ]:
merged_df.to_excel('batters_against_bowlingtype.xlsx', index=False)

In [ ]:
batters_against_rws=merged_df[merged_df['bowl_style']=='RWS'].sort_values(by='strike rate',ascending=False)
batters_against_rws.head(15)

In [ ]:
sns.scatterplot(data=batters_against_rws.iloc[:10],x='average',y='strike rate')
plt.xlabel('Average')
plt.ylabel('Strike Rate')
plt.title('Scatter Plot of Player vs Average')
plt.show()

# Bowler's performance against right and left handed batters

In [ ]:
bowl_df=df.groupby(['p_bowl','bowl','bat_hand']).agg({'bowlruns':'sum','ballfaced':'sum','p_match':'nunique','bowl_style':'max'}).reset_index()
bowl_df['economy']=bowl_df['bowlruns']*6/bowl_df['ballfaced']
bowl_df.head()

In [ ]:
bowl_outs=df[df['out']==True].groupby(['p_bowl','bowl','bat_hand']).size().reset_index(name='outs')
bowl_outs.head()

In [ ]:
merged_bowl=pd.merge(bowl_df,bowl_outs,on=['p_bowl','bowl','bat_hand'])
merged_bowl['average']=merged_bowl['bowlruns']/merged_bowl['outs']
merged_bowl['strike rate']=merged_bowl['ballfaced']/merged_bowl['outs']
merged_bowl=merged_bowl[(merged_bowl['p_match']>25) & (merged_bowl['ballfaced']>500)].sort_values(by='average', ascending=True)
merged_bowl.head(20)

In [ ]:
merged_bowl.to_excel('Bowling Performaces against left and right handers.xlsx',index=False)

In [ ]:
df[(df['p_match']=='1174798')]